# Install Library

In [ ]:
! pip install beautifulsoup4
! pip install selenium

In [1]:
# import library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Scraping Tender Data Link

In [2]:
# open website
bot = webdriver.PhantomJS(r'C:\Users\Hanif\Anaconda Projek\Botika Internship\Botika\Project\Itjip\phantomjs-2.1.1-windows\bin\phantomjs.exe')
bot.get('https://v3.opentender.net/#/tender')

# find next button of the table
btn_next = WebDriverWait(bot,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='app']/div[2]/div/div[3]/div/div[2]/div[2]/ul/li[8]/button")))

C:\Users\Hanif\anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [3]:
html = bot.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
data = bs(html, 'html.parser')
rows = data.find('table', {'class' : 'table table-bordered'})

## Jika Belum Punya Linknya

In [6]:
open_tender_link_list = []

# Scrape dibatasi per halaman
jumlah_halaman = 1
for i in range(jumlah_halaman):
    for link in rows.find_all('a', href=True):
        if 'tender-detail' in str(link):
            open_tender_link_list.append('https://opentender.net/'+link['href'])
    btn_next.click()

# Scrape semua halaman
# while btn_next.is_enabled():
#     for link in rows.find_all('a', href=True):
#         if 'tender-detail' in str(link):
#             open_tender_link_list.append('https://opentender.net/'+link['href'])
#     btn_next.click()
# Last Page
# for link in rows.find_all('a', href=True):
#         if 'tender-detail' in str(link):
#             open_tender_link_list.append('https://opentender.net/'+link['href'])

print('Proses Scraping Link Open Tender Selesai...')
print('Link yang didapatkan : ' + str(len(open_tender_link_list)) + ' data')
print('Berikut sample link yang didapatkan : ' + str(open_tender_link_list[:2]))

Proses Scraping Link Open Tender Selesai...
Link yang didapatkan : 25 data
Berikut sample link yang didapatkan : ['https://opentender.net/#/tender-detail/4925', 'https://opentender.net/#/tender-detail/56216']


In [18]:
with open('tender_link.txt', 'w+') as f:
    for link in open_tender_link_list:
        f.write("%s \n" % link)

## Jika sudah punya daftar linknya

In [19]:
link_data = open('tender_link.txt','r+').readlines()
for link in link_data:
    link = link.strip()
    open_tender_link_list.append(link)

In [21]:
open_tender_link_list[-4:]

['https://opentender.net/#/tender-detail/94097',
 'https://opentender.net/#/tender-detail/96361',
 'https://opentender.net/#/tender-detail/98060',
 'https://opentender.net/#/tender-detail/98728']

# Scrape Data per Tender

In [5]:
tender_datas = []
for open_tender in open_tender_link_list:
    print('Geting data from : ' + open_tender + ' please wait...')
    bot = webdriver.PhantomJS(r'C:\Users\Hanif\Anaconda Projek\Botika Internship\Botika\Project\Itjip\phantomjs-2.1.1-windows\bin\phantomjs.exe')
    bot.get(open_tender)
    html = bot.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    tender = bs(html, 'html.parser')
    rows = tender.findAll('table', {'class' : ['table table-sm table-striped','table table-sm table-striped col-12 col-sm-6']})
    tender_data = []
    for row in rows:
        for data in row.findAll(['td']):
            tender_data.append(data.get_text().strip())
    print('found ' + str(len(tender_data)) + ' datas in ' + open_tender)
    print(' ')
    if len(tender_data) == 22:
        tender_datas.append(tender_data)

Geting data from : https://opentender.net/#/tender-detail/4925 please wait...
found 22 datas in https://opentender.net/#/tender-detail/4925
 
Geting data from : https://opentender.net/#/tender-detail/56216 please wait...
found 22 datas in https://opentender.net/#/tender-detail/56216
 
Geting data from : https://opentender.net/#/tender-detail/59119 please wait...
found 22 datas in https://opentender.net/#/tender-detail/59119
 
Geting data from : https://opentender.net/#/tender-detail/62181 please wait...
found 22 datas in https://opentender.net/#/tender-detail/62181
 
Geting data from : https://opentender.net/#/tender-detail/65019 please wait...
found 36 datas in https://opentender.net/#/tender-detail/65019
 
Geting data from : https://opentender.net/#/tender-detail/70725 please wait...
found 22 datas in https://opentender.net/#/tender-detail/70725
 
Geting data from : https://opentender.net/#/tender-detail/71271 please wait...
found 3 datas in https://opentender.net/#/tender-detail/712

In [ ]:
print('Terdapat ' + str(len(tender_datas)) + ' data tender yang didapatkan dengan 22 data dalam tender tanpa Rancangan Usulan Pengadaan (RUP)')

In [9]:
df_tender = pd.DataFrame(tender_datas)
df_tender.columns = ['Nama_Proyek','Provinsi','Kota/Kabupaten','LPSE','SKPD','Agen','Panitia','Peserta/Penawar','Tender_Cepat','Kode_Lelang','Pengumuman','Akhir_Lelang','Tahun_Anggaran','Kategori','Sumber_Dana','Pagu','HPS','Nilai_Penawaran','Nilai_Kontrak','Nama_Pemenang','Direktur','No.NPWP']

In [12]:
df_tender.tail(3)

,Nama_Proyek,Provinsi,Kota/Kabupaten,LPSE,SKPD,Agen,Panitia,Peserta/Penawar,Tender_Cepat,Kode_Lelang,...,Tahun_Anggaran,Kategori,Sumber_Dana,Pagu,HPS,Nilai_Penawaran,Nilai_Kontrak,Nama_Pemenang,Direktur,No.NPWP
6,"Pembangunan Jalan Ruas Jalan Elat - Werka - Wetuar - Tamangil - Weduar Feer (DAK) (3,5 km)",Maluku,Maluku Tenggara (Kab.),LPSE Kabupaten Maluku Tenggara,-,-,Kelompok Kerja Pengadaan Jasa Konstruksi,2 /\n 0,Tidak,1087618,...,2018,Pekerjaan Konstruksi,APBD,"Rp 19.630.000.000,00","Rp 19.630.000.000,00","Rp 19.302.000.000,00","Rp 19.302.000.000,00",PT. EVAV BANGUN MANDIRI,-,01.699.991.4-941.000
7,Peningkatan Jalan Hotmix Ruas Jalan Sp. Elat-Weduar-Tamngil-Weduar Feer (DID),Maluku,Maluku Tenggara (Kab.),LPSE Kabupaten Maluku Tenggara,-,-,Kelompok Kerja Pengadaan Jasa Konstruksi,2 /\n 0,Tidak,1086618,...,2018,Pekerjaan Konstruksi,APBD,"Rp 25.850.000.000,00","Rp 25.850.000.000,00","Rp 25.350.000.000,00","Rp 25.350.000.000,00",PT. EVAV BANGUN MANDIRI,-,01.699.991.4-941.000
8,Konstruksi Peningkatan Jalan Menuju Bandara Long Ampung Kec. Kayan Selatan (DAK PENUGASAN),Kalimantan Timur,Malinau (Kab.),LPSE Kabupaten Malinau,-,-,POKJA I,5 /\n 0,Tidak,1989321,...,2018,Pekerjaan Konstruksi,APBD,"Rp 16.618.500.000,00","Rp 16.618.500.000,00","Rp 16.566.473.121,00","Rp 16.566.473.121,00",PT. BAHTRA INDAH JAYA,-,02.088.734.5-727.000


In [16]:
df_tender.to_csv('data_tender.csv', index=False)
df_tender.to_excel('data_tender.xlsx', index=False)